In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/articles-data/echr_dataset/README
/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_procedure.csv
/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_relevantLaw.csv
/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_law.csv
/kaggle/input/articles-data/echr_dataset/Article3/topics3.csv
/kaggle/input/articles-data/echr_dataset/Article3/topics3_vocab.txt
/kaggle/input/articles-data/echr_dataset/Article3/cases_a3.csv
/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_full.csv
/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_circumstances.csv
/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_featureNames.txt
/kaggle/input/articles-data/echr_dataset/Article8/ngrams_a8_full.csv
/kaggle/input/articles-data/echr_dataset/Article8/ngrams_a8_laws.csv
/kaggle/input/articles-data/echr_dataset/Article8/ngrams_a8_procedure.csv
/kaggle/input/articles-data/echr_dataset/Article8/topics8.csv
/kaggle/input/articles-data/echr_dataset/Ar

In [2]:
#For Article 3
file_path = "/kaggle/input/articles-data/echr_dataset/Article3/topics3_vocab.txt"
from collections import Counter 

# Step 2: Open the file and read its contents
with open(file_path, 'r') as file:
        # Read the file contents
        text = file.read()
        
        # Remove commas and split the text into words
        words = text.replace(',', ' ').split()
        
        # Step 3: Count the frequencies of each word
        word_counts = Counter(words)
        
        # Step 4: Sort by frequency in descending order and get the top 2000 words
        top_2000_words = word_counts.most_common(2000)
    
         # Step 5: Separate words and frequencies into two lists
        words_array = [word for word, frequency in top_2000_words]
        frequencies_array = [frequency for word, frequency in top_2000_words]
        
        # Print the top 2000 words and their frequencies
        #for word, frequency in top_2000_words:
            #print(f"{word}: {frequency}")


# 1. Data Loading and Preprocessing Each article folder contains:

cases_aN.csv for the labels (violation or no violation), N-gram feature files for different sections of each case, Topic representations and feature names. Start by loading the data files

In [3]:

import pandas as pd

# Load labels
labels_df = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article3/cases_a3.csv")

# Load N-gram features for each case section
ngrams_full = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_full.csv")
ngrams_procedure = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_procedure.csv")
ngrams_circumstances = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_circumstances.csv")
ngrams_relevant_law = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_relevantLaw.csv")
ngrams_law = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_law.csv")




# 2. Feature Engineering
Combine the features based on your case requirements. You might want to aggregate some sections, as specified:

Facts = ngrams_aN_circumstances + ngrams_aN_relevantLaw+ procedure+ full + law , you can try out various possibilities . Follow the guidelines which decides the most cases hence the relevant feature 

In [4]:
# Combine relevant N-gram features (e.g., "Facts" = Circumstances + Relevant Law+ procedure + full +law)
ngrams_facts = ngrams_circumstances.add(ngrams_relevant_law, fill_value=0)
ngrams_facts = ngrams_facts.add(ngrams_procedure, fill_value=0)
ngrams_facts = ngrams_facts.add(ngrams_law, fill_value=0)
ngrams_facts = ngrams_facts.add(ngrams_full, fill_value=0)


# Load topics and vocabulary files
topics_df = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article3/topics3.csv")
ngrams_feature_names = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_featureNames.txt")
topics_vocab = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article3/ngrams_a3_featureNames.txt")


# 2. Topic Modeling
The article suggests using spectral clustering on an N-gram similarity matrix to reduce dimensionality and create semantically meaningful topics.

Step-by-Step for Topic Modeling:
Compute N-gram Similarity: Use cosine similarity on the N-grams vector representation to create a similarity matrix.
Apply Spectral Clustering: Perform clustering to group similar N-grams into 30 clusters, each representing a topic. We’ll use pre-trained clusters for simplicity, but you can adapt this to generate custom clusters.

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering

# Create similarity matrix for N-grams
ngram_vectors = ngrams_full.to_numpy()  # Assuming ngrams_full contains vector representations
similarity_matrix = cosine_similarity(ngram_vectors)

# Perform spectral clustering
n_clusters = 30
spectral_clust = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=42)
clusters = spectral_clust.fit_predict(similarity_matrix)

# Add clusters as topics to topic features DataFrame
topics_df['topic'] = clusters
# Convert clusters (topics) into a DataFrame if it's not already
topics_df = pd.DataFrame(clusters, columns=['topic'])

# Concatenate topics to ngrams_facts along columns
ngrams_facts = ngrams_facts.add(topics_df, fill_value=0)

# 4. Prepare Labels
Map labels to binary values: +1 for violation and -1 for no violation.

In [6]:
# Inspect the columns of labels_df
print("Original columns in labels_df:", labels_df.columns)

# If there are more columns, select only the ones you need
# Assuming the required columns are the first two:
labels_df = labels_df.iloc[:, :2]  # Selecting the first two columns

# Rename columns
labels_df.columns = ['case_name', 'violation_status']

# Map 'v' to +1 and 'nv' to -1 in the labels DataFrame
labels_df['label'] = labels_df['violation_status'].apply(lambda x: -1 if x == 'nv' else 1)

# Combine the dataframes as before
data_df = pd.concat([features_df, labels_df['label']], axis=1).dropna()

#Feature labelling for X and y
X = data_df.drop('label', axis=1)
y = data_df['label']

# Verify shapes of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")
X = pd.get_dummies(X, drop_first=True) 

# Drop any rows with NaN values, if applicable
X = X.dropna()


# If y is a Pandas Series
count = pd.Series(y).value_counts()
print("Number of 1's:", count.get(1))
print("Number of -1's:", count.get(-1))

#Check whether the set is imbalanced or not

# Get counts of all unique values
counts = y.value_counts()
print(counts)
print("Balanced Set")

Original columns in labels_df: Index(['CASE OF SHISHKOV v. RUSSIA', 'v'], dtype='object')


NameError: name 'features_df' is not defined

# 5. Train/Test Split and Model Training
Perform a stratified 10-fold cross-validation using a linear SVM. Tune the regularization parameter $C$ with grid search.

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
import numpy as np

# Initialize SVM model
svm_model = SVC()

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Use StratifiedKFold with n_splits=5 to ensure class distribution is maintained
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Verify class distribution in each fold
for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    y_train_fold = y[train_idx]
    unique_classes, counts = np.unique(y_train_fold, return_counts=True)
    print(f"Fold {fold_idx + 1} class distribution:", dict(zip(unique_classes, counts)))

# Initialize GridSearchCV with StratifiedKFold
grid_search = GridSearchCV(svm_model, param_grid, cv=cv, scoring='accuracy', error_score='raise')

# Fit the model with grid search
grid_search.fit(X, y)

# Best model and score
best_model = grid_search.best_estimator_
print(f"Best Score: {grid_search.best_score_}")


# Similarly for Article 6

In [ ]:
import pandas as pd

# Load labels
labels_df = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article6/cases_a6.csv")

# Load N-gram features for each case section
ngrams_full = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article6/ngrams_a6_full.csv")
ngrams_procedure = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article6/ngrams_a6_procedure.csv")
ngrams_circumstances = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article6/ngrams_a6_circumstances.csv")
ngrams_relevant_law = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article6/ngrams_a6_relevantLaw.csv")
ngrams_law = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article6/ngrams_a6_law.csv")

# Combine relevant N-gram features (e.g., "Facts" = Circumstances + Relevant Law+ procedure + full +law)
ngrams_facts = ngrams_circumstances.add(ngrams_relevant_law, fill_value=0)
ngrams_facts = ngrams_facts.add(ngrams_procedure, fill_value=0)
ngrams_facts = ngrams_facts.add(ngrams_law, fill_value=0)
ngrams_facts = ngrams_facts.add(ngrams_full, fill_value=0)
# Load topics and vocabulary files
topics_df = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article6/topics6.csv")
ngrams_feature_names = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article6/ngrams_a6_featureNames.txt")
topics_vocab = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article6/ngrams_a6_featureNames.txt")
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering

# Create similarity matrix for N-grams
ngram_vectors = ngrams_full.to_numpy()  # Assuming ngrams_full contains vector representations
similarity_matrix = cosine_similarity(ngram_vectors)

# Perform spectral clustering
n_clusters = 30
spectral_clust = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=42)
clusters = spectral_clust.fit_predict(similarity_matrix)

# Add clusters as topics to topic features DataFrame
topics_df['topic'] = clusters
# Convert clusters (topics) into a DataFrame if it's not already
topics_df = pd.DataFrame(clusters, columns=['topic'])

# Concatenate topics to ngrams_facts along columns
ngrams_facts = pd.concat([ngrams_facts, topics_df], axis=1)
# Inspect the columns of labels_df
print("Original columns in labels_df:", labels_df.columns)

# If there are more columns, select only the ones you need
# Assuming the required columns are the first two:
labels_df = labels_df.iloc[:, :2]  # Selecting the first two columns

# Rename columns
labels_df.columns = ['case_name', 'violation_status']

# Map 'v' to +1 and 'nv' to -1 in the labels DataFrame
labels_df['label'] = labels_df['violation_status'].apply(lambda x: -1 if x == 'nv' else 1)

# Combine the dataframes as before
data_df = pd.concat([features_df, labels_df['label']], axis=1).dropna()

#Feature labelling for X and y
X = data_df.drop('label', axis=1)
y = data_df['label']

# Verify shapes of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")
X = pd.get_dummies(X, drop_first=True) 

# Drop any rows with NaN values, if applicable
X = X.dropna()


# If y is a Pandas Series
count = pd.Series(y).value_counts()
print("Number of 1's:", count.get(1))
print("Number of -1's:", count.get(-1))

# Get counts of all unique values
counts = y.value_counts()
print(counts)
print("Balanced Set")
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
import numpy as np

# Initialize SVM model
svm_model = SVC()

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Use StratifiedKFold with n_splits=5 to ensure class distribution is maintained
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Verify class distribution in each fold
for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    y_train_fold = y[train_idx]
    unique_classes, counts = np.unique(y_train_fold, return_counts=True)
    print(f"Fold {fold_idx + 1} class distribution:", dict(zip(unique_classes, counts)))

# Initialize GridSearchCV with StratifiedKFold
grid_search = GridSearchCV(svm_model, param_grid, cv=cv, scoring='accuracy', error_score='raise')

# Fit the model with grid search
grid_search.fit(X, y)

# Best model and score
best_model = grid_search.best_estimator_
print(f"Best Score: {grid_search.best_score_}")



# Similarly for Article-8

In [ ]:
import pandas as pd

# Load labels
labels_df = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article8/cases_a8.csv")

# Load N-gram features for each case section
ngrams_full = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article8/ngrams_a8_full.csv")
ngrams_procedure = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article8/ngrams_a8_procedure.csv")
ngrams_circumstances = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article8/ngrams_a8_circumstanes.csv")
ngrams_relevant_law = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article8/ngrams_a8_relevantLaw.csv")
ngrams_law = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article8/ngrams_a8_laws.csv")

# Combine relevant N-gram features (e.g., "Facts" = Circumstances + Relevant Law+ procedure + full +law)
ngrams_facts = ngrams_circumstances.add(ngrams_relevant_law, fill_value=0)
ngrams_facts = ngrams_facts.add(ngrams_procedure, fill_value=0)
ngrams_facts = ngrams_facts.add(ngrams_law, fill_value=0)
ngrams_facts = ngrams_facts.add(ngrams_full, fill_value=0)
# Load topics and vocabulary files
topics_df = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article8/topics8.csv")
ngrams_feature_names = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article8/ngrams_a8_featureNames.txt")
topics_vocab = pd.read_csv("/kaggle/input/articles-data/echr_dataset/Article8/ngrams_a8_featureNames.txt")
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering

# Create similarity matrix for N-grams
ngram_vectors = ngrams_full.to_numpy()  # Assuming ngrams_full contains vector representations
similarity_matrix = cosine_similarity(ngram_vectors)

# Perform spectral clustering
n_clusters = 30
spectral_clust = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=42)
clusters = spectral_clust.fit_predict(similarity_matrix)

# Add clusters as topics to topic features DataFrame
topics_df['topic'] = clusters
# Convert clusters (topics) into a DataFrame if it's not already
topics_df = pd.DataFrame(clusters, columns=['topic'])

# Concatenate topics to ngrams_facts along columns
ngrams_facts = pd.concat([ngrams_facts, topics_df], axis=1)
# Inspect the columns of labels_df
print("Original columns in labels_df:", labels_df.columns)

# If there are more columns, select only the ones you need
# Assuming the required columns are the first two:
labels_df = labels_df.iloc[:, :2]  # Selecting the first two columns

# Rename columns
labels_df.columns = ['case_name', 'violation_status']

# Map 'v' to +1 and 'nv' to -1 in the labels DataFrame
labels_df['label'] = labels_df['violation_status'].apply(lambda x: -1 if x == 'nv' else 1)

# Combine the dataframes as before
data_df = pd.concat([features_df, labels_df['label']], axis=1).dropna()

#Feature labelling for X and y
X = data_df.drop('label', axis=1)
y = data_df['label']

# Verify shapes of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")
X = pd.get_dummies(X, drop_first=True) 

# Drop any rows with NaN values, if applicable
X = X.dropna()


# If y is a Pandas Series
count = pd.Series(y).value_counts()
print("Number of 1's:", count.get(1))
print("Number of -1's:", count.get(-1))

# Get counts of all unique values
counts = y.value_counts()
print(counts)
print("Balanced Set")
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
import numpy as np

# Initialize SVM model
svm_model = SVC()

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Use StratifiedKFold with n_splits=5 to ensure class distribution is maintained
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Verify class distribution in each fold
for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    y_train_fold = y[train_idx]
    unique_classes, counts = np.unique(y_train_fold, return_counts=True)
    print(f"Fold {fold_idx + 1} class distribution:", dict(zip(unique_classes, counts)))

# Initialize GridSearchCV with StratifiedKFold
grid_search = GridSearchCV(svm_model, param_grid, cv=cv, scoring='accuracy', error_score='raise')

# Fit the model with grid search
grid_search.fit(X, y)

# Best model and score
best_model = grid_search.best_estimator_
print(f"Best Score: {grid_search.best_score_}")

